In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

圧縮されているので解凍する必要有り

In [ ]:
# !apt-get install p7zip
# !p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
# !p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z

In [ ]:
# タブ区切りのデータ
train = pd.read_csv('/kaggle/input/mercari/train.tsv', sep='\t')
test = pd.read_csv('/kaggle/input/mercari/test.tsv', sep='\t')

* train_id, test_id	ID
* name	商品名
* item_condition_id	商品のコンディション(悪い〜良いの5段階)　※1は良好～5は不良
* category_name	商品のカテゴリ (最大3つ)
* brand_name	ブランド名
* price	商品が販売された時の価格(ドル)で、今回求める数値
* shipping	送料が出品者負担かどうか(出品者負担: 1、購入者負担: 0)
* item_description	商品の説明




## 必要なライブラリをインポート

In [ ]:
import numpy as np
import pandas as pd
import scipy


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)

from datetime import datetime
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import lightgbm as lgb

# 最大カラム数を100に拡張(デフォルトだと省略されてしまうので)
# 常に全ての列（カラム）を表示
pd.options.display.max_columns = None
pd.options.display.max_rows = 80

# 小数点2桁で表示(指数表記しないように)
pd.options.display.float_format = '{:.2f}'.format
%matplotlib inline
#ワーニングを抑止
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
import string

# 出現回数がMAX_FEAT_DESCP以下の単語は排除する
MAX_FEAT_DESCP = 1000

import gc

In [ ]:
train = train[:10000]
test = test[:10000]

# Idは不要なので、削除して別に変数化し、スコア提出時に使用
train_Id = train.train_id
test_Id = test.test_id

# Id列削除
train.drop('train_id', axis=1, inplace=True)
test.drop('test_id', axis=1, inplace=True)

# 1.データの概要確認

## データのレコード数とカラム数を確認

In [ ]:
print(train.shape)
print(test.shape)

## データのデータ定義を確認

In [ ]:
train.info()

## データの数値項目の統計要約量を表示

In [ ]:
train.describe()

## オブジェクト型のレコード数、ユニーク数、最頻値の出現回数を表示

In [ ]:
train.describe(include='O')

## 欠損値を確認

In [ ]:
#isnull().sum()を使用して欠損値のカウントを表示する
print(train.isnull().sum())
print("-"*30)
print(test.isnull().sum())

## 欠損値を補完
* brand_nameとcategory_nameについては欠損値補完が必要
* brand_nameは、missingを設定
* item_descriptionは、Noneを設定

In [ ]:
def handle_missing_inplace(dataset):
    dataset['brand_name'].fillna(value='missing', inplace=True)
    dataset['item_description'].fillna(value='None', inplace=True)

handle_missing_inplace(train)
handle_missing_inplace(test)

In [ ]:
#isnull().sum()を使用して欠損値のカウントを表示する
print(train.isnull().sum())
print("-"*30)
print(test.isnull().sum())

# 2.目的変数の確認

## 目的変数であるpriceの要約統計量を表示

In [ ]:
train["price"].describe()

In [ ]:
# 分布確認
fig = plt.figure(figsize=(10, 4))
plt.subplots_adjust(wspace=0.4)

# ヒストグラム
ax = fig.add_subplot(1, 2, 1)
sns.distplot(train['price'], ax=ax)

# QQプロット
ax2 = fig.add_subplot(1, 2, 2)
stats.probplot(train['price'], plot=ax2)

plt.show()

# 変換後の要約統計量表示
print(train['price'].describe())
print("-"*30)
print("歪度: %f" % train['price'].skew())
print("尖度: %f" % train['price'].kurt())

## priceを対数変換

In [ ]:

df = train.copy()

#目的変数の対数log(x+1)をとる
df['price'] = np.log1p(df['price'])

# 標準化(平均0, 分散1)
scaler=StandardScaler()
df['price']=scaler.fit_transform(df[['price']])

# 分布確認
fig = plt.figure(figsize=(10, 4))
plt.subplots_adjust(wspace=0.4)

# ヒストグラム
ax = fig.add_subplot(1, 2, 1)
sns.distplot(df['price'], ax=ax)

# QQプロット
ax2 = fig.add_subplot(1, 2, 2)
stats.probplot(df['price'], plot=ax2)

plt.show()

# 変換後の要約統計量表示
print(df['price'].describe())
print("-"*30)
print("歪度: %f" % df['price'].skew())
print("尖度: %f" % df['price'].kurt())


del df
gc.collect()

# 3.説明変数の確認

##  データ型を確認

In [ ]:
train.dtypes.value_counts()

In [ ]:
cats = list(train.select_dtypes(include=['object']).columns)
nums = list(train.select_dtypes(exclude=['object']).columns)
print(f'categorical variables:  {cats}')
print(f'numerical variables:  {nums}')

## ユニーク数を表示

In [ ]:
train.nunique(axis=0)

## 名義変数、順序変数、連続変数に分ける

## name、category_name、brand_name、item_descriptionについて、ユニーク数が多すぎるため、加工をする必要あり。

## カテゴリ変数を処理
* カテゴリを/区切りで3分割する。

In [ ]:
def split_cat(text):
    """
    ・カテゴリを/で切り分ける
    ・データが存在しない場合は'No Label'を返す
    """
    try: return text.split("/")
    except: return ('No Label', 'No Label', 'No Label')

# 3つに切り分けたカテゴリ名を'subcat_0'、'subcat_1'、'subcat_2'に登録
train['subcat_0'], train['subcat_1'], train['subcat_2'] = \
    zip(*train['category_name'].apply(lambda x: split_cat(x)))

test['subcat_0'], test['subcat_1'], test['subcat_2'] = \
    zip(*test['category_name'].apply(lambda x: split_cat(x)))

# 元のカテゴリネームは削除
train.drop('category_name', axis=1, inplace=True)
test.drop('category_name', axis=1, inplace=True)

In [ ]:
train.head()

## brand_nameを処理
brand_nameは、欠損値が多いので、nameにて補完を行う

* ブランドリストを作成 
* missing'の商品名の単語がブランドリストに存在する場合、ブランド名の'missing'を商品名に置き換える
* 商品名がブランドリストの名前と完全に一致する場合、ブランド名を商品名に置き換える

In [ ]:
# trainとtestを縦方向に結合
full_set = pd.concat([train, test])
# full_setの'brand_name'から重複なしのブランドリスト(集合)を生成
all_brands = set(full_set['brand_name'].values)

# 'brand_name'の欠損値NaNを'missing'に置き換える
train['brand_name'].fillna(value='missing', inplace=True)
test['brand_name'].fillna(value='missing', inplace=True)

# 訓練データの'brand_name'が'missing'に一致するレコード数を取得
train_premissing = len(train.loc[train['brand_name'] == 'missing'])
# テストデータの'brand_name'が'missing'に一致するレコード数を取得
test_premissing = len(test.loc[test['brand_name'] == 'missing'])

print('train_before:',train_premissing) # 書き換える前の'missing'の数
print('test_before:',test_premissing) # 書き換える前の'missing'の数

In [ ]:
def brandfinder(line):
    """
    Parameters: line(str): ブランド名

    ・ブランド名の'missing'を商品名に置き換える:
         missing'の商品名の単語がブランドリストに存在する場合
    ・ブランド名を商品名に置き換える:
        商品名がブランドリストの名前と完全に一致する場合
    ・ブランド名をそのままにする:
        商品名がブランドリストの名前と一致しない
        商品名が'missing'だが商品名の単語がブランドリストにない
    """
    brand = line[0] # 第1要素はブランド名
    name = line[1]  # 第2要素は商品名
    namesplit = name.split(' ') # 商品名をスペースで切り分ける
    
    if brand == 'missing':  # ブランド名が'missing'と一致
        for x in namesplit: # 商品名から切り分けた単語を取り出す
            if x in all_brands:                
                return name # 単語がブランドリストに一致したら商品名を返す
    if name in all_brands:  # 商品名がブランドリストに存在すれば商品名を返す
        return name
    
    return brand            # どれにも一致しなければブランド名を返す

# ブランド名の付替えを実施
train['brand_name'] = train[['brand_name','name']].apply(brandfinder, axis = 1)
test['brand_name'] = test[['brand_name','name']].apply(brandfinder, axis = 1)


In [ ]:
# 書き換えられた'missing'の数を取得
train_found = train_premissing-len(train.loc[train['brand_name'] == 'missing'])
test_found = test_premissing-len(test.loc[test['brand_name'] == 'missing'])
print('train_before 書き換える前のmissingの数:',train_premissing) # 書き換える前の'missing'の数
print('train_after 書き換えられたmissingの数:',train_found)      # 書き換えられた'missing'の数
print('test_before 書き換える前のmissingの数:',test_premissing)  # 書き換える前の'missing'の数
print('test_after 書き換えられたmissingの数:',test_found)       # 書き換えられた'missing'の数

In [ ]:
print(train.nunique(axis=0))
print("-"*20)
print(test.nunique(axis=0))

In [ ]:
# 名義変数
nominal_list = ['shipping','subcat_0','subcat_1','subcat_2','brand_name']
# 順序変数
ordinal_list = ['item_condition_id']
# 数値変数
# num_list = ['price']

print(f'nominal_list variables:  {nominal_list}')
print(f'ordinal_list variables:  {ordinal_list}')
# print(f'num_list variables:  {num_list}')

## 名義変数の分布確認

In [ ]:
df = train

# 50分類まで表示
MAX_NUM = 50

columns = len(nominal_list)/1+1

fig = plt.figure(figsize=(40, 40))
plt.subplots_adjust(hspace=0.6, wspace=0.4)

for i in range(len(nominal_list)):
    
    top_list = df[nominal_list[i]].value_counts().index[:MAX_NUM]
    df_tmp = df.loc[df[nominal_list[i]].isin(top_list)]
    
    ax = fig.add_subplot(columns, 1, i+1)
    sns.countplot(x=nominal_list[i], data=df_tmp, ax=ax)
    plt.xticks(rotation=45)
plt.show()

## 順序変数の分布確認

In [ ]:
columns = len(ordinal_list)/2+1

fig = plt.figure(figsize=(10, 5))
plt.subplots_adjust(hspace=0.6, wspace=0.4)

for i in range(len(ordinal_list)):
    ax = fig.add_subplot(columns, 1, i+1)
    sns.countplot(x=ordinal_list[i], data=train, ax=ax)
    plt.xticks(rotation=45)
plt.show()

## 数値変数の分布確認

In [ ]:
# columns = len(num_list)/2+1

# fig = plt.figure(figsize=(10, 5))
# plt.subplots_adjust(hspace=0.6, wspace=0.4)

# for i in range(len(num_list)):
#     ax = fig.add_subplot(columns, 1, i+1)

#     train[num_list[i]].hist(ax=ax)
#     ax2 = train[num_list[i]].plot.kde(ax=ax, secondary_y=True,title=num_list[i])
#     ax2.set_ylim(0)
    
# plt.show()

## 名義変数とターゲットの関係を確認
* 100ドル以下を対象、50分類まで表示

In [ ]:
# 100ドル以下を対象とする
df = train[train['price']<100]

# 50分類まで表示
MAX_NUM = 50

columns = len(nominal_list)/1+1

fig = plt.figure(figsize=(40, 40))
plt.subplots_adjust(hspace=0.6, wspace=0.4)

for i in range(len(nominal_list)):
    ax = fig.add_subplot(columns, 1, i+1)

    top_list = df[nominal_list[i]].value_counts().index[:MAX_NUM]
    df_tmp = df.loc[df[nominal_list[i]].isin(top_list)]
    
    # 回帰の場合    
    sns.boxplot(x=nominal_list[i], y=df_tmp.price, data=df_tmp, ax=ax)
    plt.xticks(rotation=45)
    # 分類の場合
#     sns.barplot(x = nominal_list[i], y = df_tmp.price, data=df, ax=ax)
plt.show()


## 順序変数とターゲットの関係を確認
* 100ドル以下を対象

In [ ]:
# 100ドル以下を対象とする
df = train[train['price']<100]

columns = len(ordinal_list)/1+1

fig = plt.figure(figsize=(10, 10))
plt.subplots_adjust(hspace=0.6, wspace=0.4)

for i in range(len(ordinal_list)):

    ax = fig.add_subplot(columns, 1, i+1)

    # 回帰の場合    
    sns.boxplot(x=ordinal_list[i], y=df.price, data=df, ax=ax)
    plt.xticks(rotation=45)
    # 分類の場合
#     sns.barplot(x = ordinal_list[i], y = df.price, data=df, ax=ax)
plt.show()


In [ ]:
def to_categorical(dataset):
    dataset['subcat_0'] = dataset['subcat_0'].astype('category')
    dataset['subcat_1'] = dataset['subcat_1'].astype('category')
    dataset['subcat_2'] = dataset['subcat_2'].astype('category')  
    dataset['brand_name'] = dataset['brand_name'].astype('category')
    dataset['item_condition_id'] = dataset['item_condition_id'].astype('category')

print('Convert categorical')
to_categorical(train)
to_categorical(test)

In [ ]:
train.head()

## 学習デートとテストデータを集約

In [ ]:
num_train = train.shape[0]
num_test = test.shape[0]
print(num_train, num_test)

full_data = pd.concat([train, test], ignore_index=True)

del train
del test
gc.collect()

In [ ]:
# カテゴリ、ブランド、3カテゴリのテキストをラベルエンコードする
from sklearn.preprocessing import LabelEncoder

# LabelEncoderの生成
le = LabelEncoder()

# 'brand_name'のエンコード
le.fit(full_data.brand_name)
full_data.brand_name = le.transform(full_data.brand_name)

# 'subcat_0'のエンコード
le.fit(full_data.subcat_0)
full_data.subcat_0 = le.transform(full_data.subcat_0)

# 'subcat_1'のエンコード
le.fit(full_data.subcat_1)
full_data.subcat_1 = le.transform(full_data.subcat_1)

# 'subcat_2'のエンコード
le.fit(full_data.subcat_2)
full_data.subcat_2 = le.transform(full_data.subcat_2)

full_data.head()

del le
gc.collect()

## item_descriptionを処理

* WordCloudでitem_descriptionを図示
* WordCloudは、文章中で出現頻度が高い単語を複数選び出し、その頻度に応じた大きさで図示する手法。

In [ ]:
from wordcloud import WordCloud
import os
wordcloud = WordCloud(width = 2400, height = 1200).generate(" ".join(full_data.item_description.astype(str)))
plt.figure(figsize = (13, 10))
plt.imshow(wordcloud)
plt.show()

「新品」、「送料無料」、「状態良好」、「未使用」、「ビクトリアシークレット」、「無煙」、「大サイズ」、「中サイズ」、 「小さいサイズ」、「良好な状態」は、いくつかの頻繁に現れるアイテム説明テキストです。

## テキスト項目(item_description,name)について前処理を実施

* テキスト前処理に適用する手順は次の通り。


1. 句読点の削除
1. 数字の削除
1. ストップワードの削除(ストップワードとは、the, a, forなど)
1. 小文字への変更

1.句読点の削除関数

In [ ]:
from string import punctuation

# 句読点を含む英数字以外のアスキー文字
punctuation

In [ ]:
# Create a list of punctuation replacements
punctuation_symbols = []
for symbol in punctuation:
    punctuation_symbols.append((symbol, ''))

In [ ]:
import string
def remove_punctuation(sentence: str) -> str:
    return sentence.translate(str.maketrans('', '', string.punctuation))

2.数字の削除関数

In [ ]:
# Remove Digits
def remove_digits(x):
    x = ''.join([i for i in x if not i.isdigit()])
    return x

3.ストップワードの削除関数

In [ ]:
# Remove Stopwords
from nltk.corpus import stopwords

stop = stopwords.words('english')

def remove_stop_words(x):
    x = ' '.join([i for i in x.lower().split(' ') if i not in stop])
    return x

4.小文字へ変換関数

In [ ]:
# Change to LowerCase Words
def to_lower(x):
    return x.lower()

* item_description、nameにて上記のテキスト前処理を実施

In [ ]:
# Remove Digits, Punctuation, Stopwords, Converting to Lower-case for full_data
full_data.item_description = full_data.item_description.astype(str)
full_data['item_description'] = full_data['item_description'].apply(remove_digits)
full_data['item_description'] = full_data['item_description'].apply(remove_punctuation)
full_data['item_description'] = full_data['item_description'].apply(remove_stop_words)
full_data['item_description'] = full_data['item_description'].apply(to_lower)
full_data['name'] = full_data['name'].apply(remove_digits)
full_data['name'] = full_data['name'].apply(remove_punctuation)
full_data['name'] = full_data['name'].apply(remove_stop_words)
full_data['name'] = full_data['name'].apply(to_lower)
full_data.head()


## TfidfVectorizerをitem_descriptionに適用
* TfidfVectorizerとはCountVectorizerとは違い、単語の出現回数だけでなく単語のレア度も考慮する。
* 例えば「です」「ます」といったどの文章にも存在する単語だったり、英語では「a」「the」のような冠詞は出現回数が大きく、CountVectorizerではこのような単語に大きく引きずられてしまう。
* そうではなく単語の重要度に着目してベクトル化を行いたい場合に用いられる。
* TF（単語の出現頻度）とIDF（単語のレア度）のこと
* ngram_range = (1,3)は、3つの単語の組み合わせでカウントするということ


In [ ]:
tfidf_descp = TfidfVectorizer(
    min_df=5, strip_accents='unicode', lowercase =True,
    analyzer='word', token_pattern=r'\w+', ngram_range=(1, 3), use_idf=True, 
    smooth_idf=True, sublinear_tf=True, stop_words='english')

X_descp = tfidf_descp.fit_transform(full_data["item_description"])

tfidf_name = TfidfVectorizer(
    min_df=5, strip_accents='unicode', lowercase =True,
    analyzer='word', token_pattern=r'\w+', ngram_range=(1, 3), use_idf=True, 
    smooth_idf=True, sublinear_tf=True, stop_words='english')

X_name = tfidf_name.fit_transform(full_data["name"])

print(pd.DataFrame(X_descp.toarray(), columns=tfidf_descp.get_feature_names()).head())



In [ ]:
# 説明欄について、idf値(レア度)を集計する
def compute_tfidf(description):
    description = str(description)
    description.translate(string.punctuation)

    tfidf_sum=0
    words_count=0
    for w in description.lower().split():
        words_count += 1
        if w in tfidf_dict:
            tfidf_sum += tfidf_dict[w]
    
    if words_count > 0:
        return tfidf_sum/words_count
    else:
        return 0
    
    
tfidf_dict = dict(zip(tfidf_descp.get_feature_names(), tfidf_descp.idf_))
full_data['tfidf_desp'] = full_data['item_description'].apply(compute_tfidf)

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(full_data['price'], full_data['tfidf_desp'])
plt.title('Train price X item_description TF-IDF', fontsize=15)
plt.xlabel('Price', fontsize=15)
plt.ylabel('TF-IDF', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.show()

この散布図を見ると、TF-IDFの高低は、「価格」とあまり影響がないといえる。。

In [ ]:
# item_descriptionにて上位10位の変数を表示
idf = dict(zip(tfidf_descp.get_feature_names(), tfidf_descp.idf_))
df_idf_descp = pd.DataFrame(columns=['idf']).from_dict(
                    dict(idf), orient='index')
df_idf_descp.columns = ['idf']
display(df_idf_descp.sort_values("idf").head(10).T)

In [ ]:
# nameにて上位10位の変数を表示
idf = dict(zip(tfidf_descp.get_feature_names(), tfidf_name.idf_))
df_idf_name = pd.DataFrame(columns=['idf']).from_dict(
                    dict(idf), orient='index')
df_idf_name.columns = ['idf']
display(df_idf_name.sort_values("idf").head(10).T)

In [ ]:
# item_descriptionとname変数をスタック
X = scipy.sparse.hstack((X_name,
                         X_descp)).tocsr()
X_desc_name = pd.DataFrame(X.toarray())
gc.collect()

In [ ]:
# 元変数を削除
full_data.drop('name', axis=1, inplace=True)
full_data.drop('item_description', axis=1, inplace=True)

#　元のデータフレームと結合
full_data = pd.concat([full_data, X_desc_name], axis=1,sort=False)

del X_desc_name
gc.collect()

In [ ]:
full_data.head()

In [ ]:
# データを再度分割
X_train = full_data[:num_train]
X_test = full_data[num_train:]

# priceを対数変換する
y_train = np.log1p(X_train['price'])
# priceを削除
X_train.drop('price', axis=1, inplace=True)

del full_data
gc.collect()

In [ ]:
#訓練データとモデル評価用データに分けるライブラリ
from sklearn.model_selection import train_test_split

#フォールドアウト法により、学習データとテストデータに分割 
(X_train, X_valid, y_train, y_valid) = train_test_split(X_train, y_train , test_size = 0.3 , random_state = 0)

print("X_train: "+str(X_train.shape))
print("X_valid: "+str(X_valid.shape))
print("y_train: "+str(y_train.shape))
print("y_valid: "+str(y_valid.shape))

In [ ]:
#LightGBMライブラリ
import lightgbm as lgb
#ハイパーパラメータチューニング自動化ライブラリ
import optuna

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid)

In [ ]:
#Optunaで最適化されたパラメータ
params = {"metric": {'rmse'},
          "max_depth" : 7,
          "subsumple" : 0.0527053286950852,
          "subsample_freq" : 0,
          "leaning_rate" : 0.00012337315517641352,
          "feature_fraction" : 0.27094712699951107,
          "lambda_l1" : 0.4567708349707908,
          "lambda_l2" :6.452511288039886e-07
         }
 
#LightGBMのモデル構築
gbm = lgb.train(params,
                lgb_train,
                valid_sets=(lgb_train, lgb_eval),
                num_boost_round=10000,
                early_stopping_rounds=100,
                verbose_eval=50)

In [ ]:
#特徴量の重要度
lgb.plot_importance(gbm, figsize=(12, 6), max_num_features=10)
plt.show();
